In [1]:
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
pcd = o3d.io.read_point_cloud("data/bunny.pcd")

In [3]:
print(pcd)

PointCloud with 397 points.


In [4]:
import numpy as np
import open3d as o3d
from open3d.visualization import Visualizer
# from open3d.visualization import JVisualizer
# from open3d import JVisualizer


points = (np.random.rand(1000, 3) - 0.5) / 4
colors = np.random.rand(1000, 3)

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)
pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd])


In [5]:
# http://www.open3d.org/docs/0.6.0/python_api/open3d.geometry.Geometry3D.html?highlight=transform#open3d.geometry.Geometry3D.transform
cyl = o3d.geometry.TriangleMesh.create_cylinder(radius=1.0, height=2.0, resolution=20, split=4)
# dir(cyl)

In [6]:
o3d.visualization.draw_geometries([cyl])

In [7]:
# examples/python/visualization/non_blocking_visualization.py

import open3d as o3d
import numpy as np
import copy

if __name__ == "__main__":
    # 1. Load data and set model poses
    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
    source_raw = o3d.io.read_point_cloud( "data/cloud_bin_0.pcd" )
    target_raw = o3d.io.read_point_cloud( "data/cloud_bin_1.pcd" )
    source = source_raw.voxel_down_sample(voxel_size=0.02)
    target = target_raw.voxel_down_sample(voxel_size=0.02)
    trans = [ [ 0.862, 0.011, -0.507,  0.0] , 
              [-0.139, 0.967, -0.215,  0.7] ,
              [ 0.487, 0.255,  0.835, -1.4] , 
              [ 0.0  , 0.0  ,  0.0  ,  1.0] ]
    source.transform(trans)

    flip_transform = [ [ 1,  0,  0, 0 ] , 
                       [ 0, -1,  0, 0 ] , 
                       [ 0,  0, -1, 0 ] , 
                       [ 0,  0,  0, 1 ] ]
    
    source.transform( flip_transform )
    target.transform( flip_transform )

    # 2. Init vis window
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(source)
    vis.add_geometry(target)
    threshold = 0.05
    icp_iteration = 100
    save_image = False

    # 3. Animation loop
    for i in range( icp_iteration ):
        reg_p2l = o3d.pipelines.registration.registration_icp(
            source, target, threshold, np.identity(4),
            o3d.pipelines.registration.TransformationEstimationPointToPlane(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=1))
        source.transform(reg_p2l.transformation)
        vis.update_geometry(source)
        vis.poll_events()
        vis.update_renderer()
        if save_image:
            vis.capture_screen_image( "temp_%04d.jpg" % i )
            
    # 4. Loop over, destroy window
    vis.destroy_window()

[Open3D DEBUG] Format auto File data/cloud_bin_0.pcd
[Open3D DEBUG] PCD header indicates 8 fields, 32 bytes per point, and 198835 points in total.
[Open3D DEBUG] x, F, 4, 1, 0
[Open3D DEBUG] y, F, 4, 1, 4
[Open3D DEBUG] z, F, 4, 1, 8
[Open3D DEBUG] rgb, F, 4, 1, 12
[Open3D DEBUG] normal_x, F, 4, 1, 16
[Open3D DEBUG] normal_y, F, 4, 1, 20
[Open3D DEBUG] normal_z, F, 4, 1, 24
[Open3D DEBUG] curvature, F, 4, 1, 28
[Open3D DEBUG] Compression method is 1.
[Open3D DEBUG] Points: yes;  normals: yes;  colors: yes
[Open3D DEBUG] Read geometry::PointCloud: 198835 vertices.
[Open3D DEBUG] [RemoveNonFinitePoints] 0 nan points have been removed.
[Open3D DEBUG] Format auto File data/cloud_bin_1.pcd
[Open3D DEBUG] PCD header indicates 8 fields, 32 bytes per point, and 137833 points in total.
[Open3D DEBUG] x, F, 4, 1, 0
[Open3D DEBUG] y, F, 4, 1, 4
[Open3D DEBUG] z, F, 4, 1, 8
[Open3D DEBUG] rgb, F, 4, 1, 12
[Open3D DEBUG] normal_x, F, 4, 1, 16
[Open3D DEBUG] normal_y, F, 4, 1, 20
[Open3D DEBUG] norm